In [ ]:
# default_exp data_cleaning_linkedin_jobs

# data cleaning for linkedin jobs

> API details.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
# from nbdev.showdoc import *

In [ ]:
import re
import string
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
import config


In [ ]:
# Load data of linkedin jobs
jobs = pd.read_csv(config.LINKEDIN_JOBS, low_memory=False)
jobs.shape

(109, 5)

In [ ]:
jobs.head()

,title,company_url,published,description,criteria
0,Data Scientist,https://www.linkedin.com/company/johnson-&-joh...,2021-05-07,Job DescriptionDATA SCIENTIST - JANSSEN ANZWor...,"{'Seniority level': ['Not Applicable'], 'Emplo..."
1,Data Scientist,https://uk.linkedin.com/company/hays?trk=publi...,2021-05-06,Centrally located in Brisbane CBDMultiple work...,"{'Seniority level': ['Entry level'], 'Employme..."
2,Data Scientist - Artificial Intelligence/Machi...,https://au.linkedin.com/company/systemize-cons...,2021-05-07,"· $83,000 (inc. super) + up to 25% perf...",{'Employment type': ['Full-time']}
3,Data Scientist,https://au.linkedin.com/company/kpmg-australia...,2021-05-03,Digital Delta Data Scientist Do you love solvi...,"{'Seniority level': ['Not Applicable'], 'Emplo..."
4,Entry level Data Scientist / Risk Analyst oppo...,https://au.linkedin.com/company/commonwealthba...,2021-05-07,Entry level Data Scientist / Risk Analyst oppo...,"{'Seniority level': ['Entry level'], 'Employme..."


### Clean descriptions

In [ ]:
def clean_text(text):
    """Make text lower case, remove punctuation"""
    text = text.lower()
    text = re.sub('\xa0', ' ', text)
    text = re.sub('\u202f', ' ', text)
    text = re.sub('[0-9]', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    return text

jobs.description = jobs.description.apply(lambda x: clean_text(x))

### Remove stop words

In [ ]:
stop_words = text.ENGLISH_STOP_WORDS
def remove_stop_words(text):
    for word in stop_words:
        text = re.sub(f' {word} ', ' ', text)
    return text

jobs.description = jobs.description.apply(lambda x: remove_stop_words(x))

### Unify job titles

In [ ]:
jobs.title.value_counts().head(30)

Data Engineer                                                19
Data Scientist                                               15
Data Analyst                                                 12
Entry level Data Scientist / Risk Analyst opportunities       4
Machine Learning Engineer                                     4
Data Scientist - QuantumBlack                                 3
Data Science Engineer                                         3
Data Analyst, ANZ                                             3
Data Scientist, Customer Analytics                            3
Data Scientist - Artificial Intelligence/Machine Learning     3
Business Analyst                                              3
*Machine Learning Engineer*                                   2
Investment Data Analyst                                       2
Principal Data Scientist                                      2
Datascientist                                                 2
Data Scientist- Actuarial               

In [ ]:
official_titles = ["Data Engineer", "Data Science Engineer", "Data Warehouse",
                   "Data Scientist", "Machine Learning Engineer", "Machine Learning Specialist", "Machine Learning",
                   "Data Analyst", "Business Analyst", "Data & Analytics",                    
                   "Software Engineer"]

def unify_job_title(text):
    res = "Not sure: " + text
    for title in official_titles:
        if title in text:
            res = title
    return res

In [ ]:
jobs.loc[:,'title'] = jobs.title.apply(lambda x: unify_job_title(x))

In [ ]:
jobs.title.value_counts()

Data Scientist                                                        29
Data Engineer                                                         23
Data Analyst                                                          19
Machine Learning                                                      12
Business Analyst                                                       5
Software Engineer                                                      4
Data Science Engineer                                                  3
Not sure: Data engineer                                                2
Not sure: Data Science - Specialist                                    2
Not sure: Datascientist                                                2
Not sure: Insights and Data Science Analysts                           1
Not sure: Amazon Tech U Graduate Program 2021                          1
Not sure: Data & Reporting Analyst                                     1
Not sure: Applied Scientist - Intern               

In [ ]:
# Mapping titles
title_map = {"Data Engineer":"Data Engineer", "Data Science Engineer":"Data Engineer", "Data Warehouse":"Data Engineer",
               "Data Scientist":"Data Scientist", "Machine Learning Engineer":"Machine Learning Engineer", 
               "Machine Learning Specialist":"Machine Learning Engineer", "Machine Learning":"Machine Learning Engineer",
               "Data Analyst":"Data Analyst", "Business Analyst":"Data Analyst", "Data & Analytics":"Data Analyst",                    
               "Software Engineer":"Software Engineer"}

In [ ]:
jobs.title = jobs.title.map(title_map)

In [ ]:
jobs.isna().sum()

title          14
company_url     0
published       0
description     0
criteria        0
dtype: int64

In [ ]:
jobs.dropna(inplace=True)

In [ ]:
jobs.head()

,title,company_url,published,description,criteria
0,Data Scientist,https://www.linkedin.com/company/johnson-&-joh...,2021-05-07,job descriptiondata scientist janssen anzwor...,"{'Seniority level': ['Not Applicable'], 'Emplo..."
1,Data Scientist,https://uk.linkedin.com/company/hays?trk=publi...,2021-05-06,centrally located brisbane cbdmultiple work st...,"{'Seniority level': ['Entry level'], 'Employme..."
2,Machine Learning Engineer,https://au.linkedin.com/company/systemize-cons...,2021-05-07,· super performance bo...,{'Employment type': ['Full-time']}
3,Data Scientist,https://au.linkedin.com/company/kpmg-australia...,2021-05-03,digital delta data scientist love solving comp...,"{'Seniority level': ['Not Applicable'], 'Emplo..."
4,Data Scientist,https://au.linkedin.com/company/commonwealthba...,2021-05-07,entry level data scientist risk analyst oppo...,"{'Seniority level': ['Entry level'], 'Employme..."


In [ ]:
# Save clean data
jobs.to_csv(config.LINKEDIN_JOBS_CLEAN, index=False)

### Create Document-Term Matrix

In [ ]:
jobs = jobs.groupby(['title']).description.apply(lambda x: ','.join(x)).reset_index()

In [ ]:
cvec = CountVectorizer(stop_words='english')
data_cvec = cvec.fit_transform(jobs.description)
data_dtm = pd.DataFrame(data_cvec.toarray(), columns=cvec.get_feature_names())
data_dtm.index = jobs.title

In [ ]:
data_dtm.to_csv(config.LINKEDIN_JOBS_DTM)

In [ ]:
data_dtm

,aa,abilities,abilitiespreferred,abilitiesshow,ability,able,aboriginal,abstraction,academic,accelerating,...,youyou,zealand,zealandextraordinary,zendeks,zendesk,zero,zip,zipster,zoo,zshow
title,,,,,,,,,,,,,,,,,,,,,
Data Analyst,0,0,1,1,39,7,7,0,0,0,...,0,1,0,0,0,1,0,0,4,0
Data Engineer,0,0,0,0,22,11,3,1,2,0,...,1,1,1,0,0,0,1,0,0,0
Data Scientist,0,5,0,0,16,8,5,0,9,5,...,2,6,0,0,0,0,12,3,0,0
Machine Learning Engineer,2,0,0,0,10,1,0,0,1,0,...,0,0,0,2,28,0,0,0,0,0
Software Engineer,0,1,0,0,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
